In [1]:
import random
import dbutils
from datetime import date
from time import mktime

main_connector = dbutils.connect_to_main_database()

/data/anaconda/envs/datathon/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [15]:
#data_table = "image_data"
data_table = "pointcloud_data"

# Getting a specific date in time as a 13-digit timestamp.
year = 2018
month = 12
day = 1
start = date(year, month, day)
timestamp = str(1000 * int(mktime(start.timetuple())))

# The values of the measurents-table that should appear in the results.
values = ["qrcode", "height_cms", "weight_kgs"]
values = ["measurements." + value for value in values]
values = ", ".join(values)

# Compose an SQL-statement.
sql_statement = ""
sql_statement += "SELECT {}.path, {}".format(data_table, values)
sql_statement += " FROM {}".format(data_table)
sql_statement += " INNER JOIN measurements ON {}.measurement_id=measurements.id".format(data_table)
sql_statement += " WHERE measurements.type=\'manual\'"
sql_statement += " AND measurements.timestamp > {}".format(timestamp)
sql_statement += ";"
print(sql_statement)

# Retrieve the results.
results = main_connector.execute(sql_statement, fetch_all=True)
if len(results) == 0:
    print("No results found after {}!".format(start))
else:
    print("Found {} samples after {}.".format(len(results), start))
    
    # Turn everything into a dictionary.
    qrcodes_dictionary = {}
    for path, qrcode, height, weight in results:
        if qrcode not in qrcodes_dictionary:
            qrcodes_dictionary[qrcode] = []
        qrcodes_dictionary[qrcode].append((height, weight, path))
    
    print(results[0])
    for qrcode, samples in qrcodes_dictionary.items():
            print("QR-code {} has {} samples.".format(qrcode, len(samples)))

SELECT pointcloud_data.path, measurements.qrcode, measurements.height_cms, measurements.weight_kgs FROM pointcloud_data INNER JOIN measurements ON pointcloud_data.measurement_id=measurements.id WHERE measurements.type='manual' AND measurements.timestamp > 1543622400000;
Found 443 samples after 2018-12-01.
('/whhdata/person/MH_WHH_0013/measurements/1545379607703/pc/pc_MH_WHH_0013_1545379607703_200_009.pcd', 'MH_WHH_0013', 70.0, 7.1)
QR-code MH_WHH_0013 has 30 samples.
QR-code MH_WHH_0770 has 27 samples.
QR-code MH_WHH_1020 has 208 samples.
QR-code MH_WHH_1021 has 43 samples.
QR-code MH_WHH_1022 has 55 samples.
QR-code MH_WHH_1025 has 33 samples.
QR-code MH_WHH_1035 has 47 samples.


In [ ]:
import glob
import os

models_root_path = "/whhdata/output"
model_paths = glob.glob(os.path.join(models_root_path, "*.p"))
print("Found {} models".format(len(model_paths)))

batch_size = 128

for model_path in model_paths:
    print("Evaluating model \"{}\"...".format(model_path))
    for batch_index, index in enumerate(range(0, len(results), batch_size)):
        print(batch_index, index, index + batch_size)
